In [3]:
import os
import sys
import re
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
from larval_gonad.notebook import Nb

In [4]:
# Setup notebook
nbconfig = Nb.setup_notebook(seurat_dir='../output/scrnaseq-wf/scrnaseq_combine_force')

last updated: 2018-11-16 
Git hash:


In [5]:
cluster_annot = {
    0: 'LS',
    1: 'MC',
    2: 'MS',
    3: 'ES',
    4: 'LC',
    5: 'EC',
    6: 'SP',
    7: 'TE',
    8: 'PC',
    9: 'U9',
    10: 'U10',
    11: 'U11'
}

cluster_order = [
'SP',
'ES',
'MS',
'LS',
'EC',
'MC',
'LC',
'TE',
'PC',
]

clusters = nbconfig.seurat.get_clusters('res.0.6').map(cluster_annot)
clusters = clusters[clusters.isin(cluster_order)]
clusters = clusters.astype('category').cat.as_ordered()
clusters = clusters.cat.reorder_categories(cluster_order)
clusters.index.name = 'cell_id'

In [6]:
raw = nbconfig.seurat.get_raw()

In [7]:
def make_sheet(g, dd):
    # merge on gene symbol
    _dat = dd.T.join(pd.Series(nbconfig.fbgn2symbol, name='gene'))
    _dat.drop('cluster', inplace=True)
    _dat.set_index('gene', append=True, inplace=True)

    # make flag on
    is_on = _dat.sum(axis=1) > 0
    _dat = _dat[is_on].copy()
    
    # Write to a work sheet
    _dat.to_csv(f'../output/notebook/2018-11-16_{g}.tsv', sep='\t')

In [8]:
for g, dd in raw.T.join(clusters).groupby('cluster'):
    make_sheet(g, dd)